In [1]:
#RUN
from pandas import DataFrame
import pandas as pd
import numpy as np

In [2]:
#RUN
try:
    from urllib import urlencode
except ImportError:
    from urllib.parse import urlencode
    
from jsonapi_client import Session, Filter
import pycurl

API_BASE = 'https://www.ebi.ac.uk/metagenomics/api/latest/'

In [144]:
#RUN
# 'studies' bisa diganti dengan target semisal 'sample' ; 'MGYS00001939' merupakan accession
with Session(API_BASE) as s:
    std = s.get('studies', 'MGYS00000705').resource

In [145]:
#RUN
#memunculkan study_name
std

<ResourceObject: studies: MGYS00000705 (2296035867840) (2296035867840)>

In [146]:
#RUN
#melihat jumlah sample pada study di atas
std.samples_count

26

In [147]:
#RUN/SKIP
sample = std.samples
#melihat informasi pada salah satu sample yang dipilih
x=sample[0]
x.sample_metadata

[{'key': 'investigation type', 'unit': None, 'value': 'metagenome'},
 {'key': 'project name',
  'unit': None,
  'value': 'Effects of water-saving irrigation on the emissions of greenhouse gases and the prokaryotic communities in rice paddy soils'},
 {'key': 'geographic location (longitude)',
  'unit': None,
  'value': '37.27442N, 126.99325W'},
 {'key': 'geographic location (depth)', 'unit': None, 'value': '15 cm'},
 {'key': 'geographic location (country and/or sea,region)',
  'unit': None,
  'value': 'South Korea: Suwon'},
 {'key': 'collection date', 'unit': None, 'value': '02-Aug-2012'},
 {'key': 'environment (biome)', 'unit': None, 'value': 'Rice paddy soils'},
 {'key': 'environment (feature)', 'unit': None, 'value': 'rice field'},
 {'key': 'environment (material)', 'unit': None, 'value': 'soil'},
 {'key': 'environmental package',
  'unit': None,
  'value': 'MIGS/MIMS/MIMARKS.soil'},
 {'key': 'elevation', 'unit': None, 'value': '89 m'},
 {'key': 'NCBI sample classification', 'unit': 

In [ ]:
#SKIP
def get_metadata(metadata, key):
    import html
    for m in metadata:
        if m['key'].lower() == key.lower():
            value = m['value']
            unit = html.unescape(m['unit']) if m['unit'] else ""
            return "{value} {unit}".format(value=value, unit=unit)
    return None

biome_label = 'environment (biome)'
elev_label = 'elevation'
df = DataFrame(columns=('Sample Name', 'Longitude', 'Latitude', 'Elevation', 'Biome', 'Country'))
df.index.name = 'ID Sample'

with Session(API_BASE) as s:
    params = {
        'study_accession': 'MGYS00003963',
        'page_size': 100,
    }
    f = Filter(urlencode(params))
    for sample in s.iterate('samples', f):
        for a in sample.runs:
        #print('runs: '+str(a.accession))
            for b in a.analyses:
                print('analyses: '+str(b.accession))
        df.loc[sample.accession] = [
            sample.sample_name,
            sample.longitude,
            sample.latitude,
            get_metadata(sample.sample_metadata, elev_label),
            get_metadata(sample.sample_metadata, biome_label),
            sample.geo_loc_name
        ]
df

In [148]:
#RUN
def get_metadata(metadata, key):
    import html
    for m in metadata:
        if m['key'].lower() == key.lower():
            value = m['value']
            unit = html.unescape(m['unit']) if m['unit'] else ""
            return "{value} {unit}".format(value=value, unit=unit)
    return None

#biome_label = 'environment (biome)'
#elev_label = 'elevation'
#df = DataFrame(columns=('Sample Name', 'Longitude', 'Latitude', 'Elevation', 'Biome', 'Country'))
#df.index.name = 'ID Sample'
df = pd.DataFrame([])
acc = []
sample_name = []
analyze = []

with Session(API_BASE) as s:
    params = {
        'study_accession': 'MGYS00000705',
        'page_size': 100,
    }
    f = Filter(urlencode(params))
    for sample in s.iterate('samples', f):
        acc.append(sample.accession)
        sample_name.append(sample.sample_name)
        for a in sample.runs:
        #print('runs: '+str(a.accession))
            for b in a.analyses:
                print('analyses: '+str(b.accession))
        analyze.append(b.accession)
        meta = sample.sample_metadata
        meta = pd.DataFrame.from_dict(meta)
        meta = meta.set_index('key') 
        meta = meta.drop(columns='unit')
        meta = meta.transpose()
        df = df.append(meta)
acc = pd.DataFrame(acc, columns=['ID_Sample'])
sample_name = pd.DataFrame(sample_name, columns=['Sample Name'])
analyze = pd.DataFrame(analyze, columns=['ID_Analyses'])

analyses: MGYA00038067
analyses: MGYA00038062
analyses: MGYA00038061
analyses: MGYA00038064
analyses: MGYA00038063
analyses: MGYA00038065
analyses: MGYA00038066
analyses: MGYA00038046
analyses: MGYA00038047
analyses: MGYA00038049
analyses: MGYA00038050
analyses: MGYA00038048
analyses: MGYA00038077
analyses: MGYA00038076
analyses: MGYA00038079
analyses: MGYA00038078
analyses: MGYA00038081
analyses: MGYA00038080
analyses: MGYA00038057
analyses: MGYA00038058
analyses: MGYA00038059
analyses: MGYA00038060
analyses: MGYA00038053
analyses: MGYA00038054
analyses: MGYA00038055
analyses: MGYA00038056
analyses: MGYA00038051
analyses: MGYA00038052
analyses: MGYA00038071
analyses: MGYA00038070
analyses: MGYA00038069
analyses: MGYA00038068
analyses: MGYA00038075
analyses: MGYA00038074
analyses: MGYA00038073
analyses: MGYA00038072


In [149]:
df

,NCBI sample classification,collection date,elevation,environment (biome),environment (feature),environment (material),environmental package,"geographic location (country and/or sea,region)",geographic location (depth),geographic location (longitude),instrument model,investigation type,project name
value,410658,02-Aug-2012,89 m,Rice paddy soils,rice field,soil,MIGS/MIMS/MIMARKS.soil,South Korea: Suwon,15 cm,"37.27442N, 126.99325W",454 GS FLX Titanium,metagenome,Effects of water-saving irrigation on the emis...
value,410658,02-Aug-2012,89 m,Rice paddy soils,rice field,soil,MIGS/MIMS/MIMARKS.soil,South Korea: Suwon,15 cm,"37.27442N, 126.99325W",454 GS FLX Titanium,metagenome,Effects of water-saving irrigation on the emis...
value,410658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,454 GS FLX Titanium,NaN,NaN
value,410658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,454 GS FLX Titanium,NaN,NaN
value,410658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,454 GS FLX Titanium,NaN,NaN
value,410658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,454 GS FLX Titanium,NaN,NaN
value,410658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,454 GS FLX Titanium,NaN,NaN
value,410658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,454 GS FLX Titanium,NaN,NaN
value,410658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,454 GS FLX Titanium,NaN,NaN
value,410658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,454 GS FLX Titanium,NaN,NaN


In [ ]:
#SKIP
with Session(API_BASE) as s:
    params = {
        'study_accession': 'MGYS00003963',
        'page_size': 100,
    }
    f = Filter(urlencode(params))
    for sample in s.iterate('samples', f):
        for a in sample.runs:
        #print('runs: '+str(a.accession))
            for b in a.analyses:
                print('analyses: '+str(b.accession))

In [ ]:
#RUN/SKIP
#memunculkan list sample sesuai jumlah
std.samples

In [ ]:
#SKIP
blank = []
for x in sample:
    #print(x.sample_metadata)
    df = pd.DataFrame(x.sample_metadata)
    df['key'].tolist()
    print(df)
    #df = pd.DataFrame(x.sample_metadata)
    #print(df)
    #index = np.where(df['key'].str.contains('location'))[0]
    #blank.append(df.iloc[index, :])
    #index = np.where(df['key'].str.contains('biome'))[0]
    #blank.append(df.iloc[index, :])
    #index = np.where(df['key'].str.contains('elevation'))[0]
    #blank.append(df.iloc[index, :])
    #df_new = pd.concat(blank)
#df_new

In [ ]:
#RUN
with Session(API_BASE) as s:
    std = s.get('studies', 'MGYS00000837').resource
print(std.accession)
#melihat informasi pada salah satu sample yang dipilih
#sample = std.samples

In [ ]:
#SKIP
for x in sample:
    print('sample: '+str(x.accession))
    for a in x.runs:
        #print('runs: '+str(a.accession))
        for b in a.analyses:
            print('analyses: '+str(b.accession))

In [ ]:
#SKIP
#membuat container kosong di bawah :
df = pd.DataFrame([])
acc = []
sample_name = []
analyze = []
for x in sample:
    acc.append(x.accession)
    sample_name.append(x.sample_name)
    for a in x.runs:
        #print('runs: '+str(a.accession))
        for b in a.analyses:
            print('analyses: '+str(b.accession))
    analyze.append(b.accession)
    meta = x.sample_metadata
    meta = pd.DataFrame.from_dict(meta)
    meta = meta.set_index('key') 
    meta = meta.drop(columns='unit')
    meta = meta.transpose()
    df = df.append(meta)
acc = pd.DataFrame(acc, columns=['ID_Sample'])
sample_name = pd.DataFrame(sample_name, columns=['Sample Name'])
analyze = pd.DataFrame(analyze, columns=['ID_Analyses'])

In [150]:
#RUN
#reset index agar dapat di merge
df = df.reset_index()
df = df.drop(columns='index')
#df = df.drop(columns='level_0')

In [151]:
#RUN
#script merge
merged = acc.merge(sample_name, left_index=True, right_index=True, how='inner')
merged = merged.merge(analyze, left_index=True, right_index=True, how='inner')
merged = merged.merge(df, left_index=True, right_index=True, how='inner')
#df
merged

,ID_Sample,Sample Name,ID_Analyses,NCBI sample classification,collection date,elevation,environment (biome),environment (feature),environment (material),environmental package,"geographic location (country and/or sea,region)",geographic location (depth),geographic location (longitude),instrument model,investigation type,project name
0,SRS462742,MIMS sample from soil metagenome,MGYA00038065,410658,02-Aug-2012,89 m,Rice paddy soils,rice field,soil,MIGS/MIMS/MIMARKS.soil,South Korea: Suwon,15 cm,"37.27442N, 126.99325W",454 GS FLX Titanium,metagenome,Effects of water-saving irrigation on the emis...
1,SRS462743,MIMS sample from soil metagenome,MGYA00038048,410658,02-Aug-2012,89 m,Rice paddy soils,rice field,soil,MIGS/MIMS/MIMARKS.soil,South Korea: Suwon,15 cm,"37.27442N, 126.99325W",454 GS FLX Titanium,metagenome,Effects of water-saving irrigation on the emis...
2,SRS346911,Bacterial sequences obtained from the O plot i...,MGYA00038077,410658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,454 GS FLX Titanium,NaN,NaN
3,SRS346912,bacterial sequences obtained from the APK plot...,MGYA00038076,410658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,454 GS FLX Titanium,NaN,NaN
4,SRS346913,bacterial sequences obtained from the CAPK plo...,MGYA00038079,410658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,454 GS FLX Titanium,NaN,NaN
5,SRS346914,bacterial sequences obtained from the LAPK plo...,MGYA00038078,410658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,454 GS FLX Titanium,NaN,NaN
6,SRS346915,bacterial sequences obtained from the WAPK plo...,MGYA00038081,410658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,454 GS FLX Titanium,NaN,NaN
7,SRS346916,bacterial sequences obtained from the CLWAPK p...,MGYA00038080,410658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,454 GS FLX Titanium,NaN,NaN
8,SRS346917,bacterial sequences obtained from the O plot i...,MGYA00038057,410658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,454 GS FLX Titanium,NaN,NaN
9,SRS346918,bacterial sequences obtained from the APK plot...,MGYA00038058,410658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,454 GS FLX Titanium,NaN,NaN


In [51]:
#RUN SEKALI AJA
#membuat folder
from pathlib import Path
import os
#_dir = os.getcwd()
_dir = 'D:\\Seminar\\Seminar_Anam'
_dir = os.path.join(_dir, 'Study_Padi')
if not os.path.exists(_dir):
    os.makedirs(_dir)
print(_dir)

D:\Seminar\Seminar_Anam\Study_Padi


In [152]:
#RUN
#Set default cwd
#os.chdir('D:\\Seminar\\Seminar_Anam')
#print(os.getcwd())
#masuk ke folder "Study_Jagung/Padi"
#%cd Study_Padi
#export ke .csv
merged.to_csv(std.accession+'.csv')

In [ ]:
#SKIP
for x in sample:
    #print('sample: '+str(i.accession))
    for a in x.runs:
        #print('runs: '+str(a.accession))
        for b in a.analyses:
            print('analyses: '+str(b.accession))

In [ ]:
#SKIP
#memasukkan sample yang dipilih ke bentuk kolom
df = pd.DataFrame(sample[0].sample_metadata)
df

In [ ]:
#SKIP
blank = []

In [ ]:
#SKIP
#membuat index baru dari list 'key' yang dipilih
index = np.where(df['key'].str.contains('location'))[0]
blank.append(df.iloc[index, :])
index = np.where(df['key'].str.contains('biome'))[0]
blank.append(df.iloc[index, :])
index = np.where(df['key'].str.contains('elevation'))[0]
blank.append(df.iloc[index, :])

In [ ]:
#SKIP
#'key' yang dilih dijadikan data frame baru
df_new = pd.concat(blank)
df_new

In [ ]:
#SKIP
filename_sample = '' 

In [ ]:
#SKIP
#membuat loop accession di dalam bentuk list
for x in sample:
    #print('sample: '+str(i.accession))
    for a in x.runs:
        #print('runs: '+str(a.accession))
        for b in a.analyses:
            print('analyses: '+str(b.accession))
            #for z in y.downloads:
                #if 'json.biom' in z.url:
                    #print(z.url)

In [ ]:
#SKIP
#membuat folder
from pathlib import Path
import os
_dir = os.getcwd()
#_dir = 'D:\\Seminar\\Seminar_Anam'
_dir = os.path.join(_dir, 'Study_Jagung')
if not os.path.exists(_dir):
    os.makedirs(_dir)
print(_dir)
#%rmdir(Study+str)
#with open('_dir'+i.accession+'.biom', 'wb') as f:
    #c = pycurl.Curl()
    #c.setopt(c.URL, z.url)
    #c.setopt(c.WRITEDATA, f)
    #c.perform()
#c.close()